In [1]:
%matplotlib inline

In [2]:
from keras.models import Model
from keras.layers import Input,  Dense, LSTM, GRU
from keras.layers import Embedding

from keras.utils import plot_model

Using TensorFlow backend.


# Encoder

In [3]:
latent_dim = 512

In [4]:
def connect_encoder():
    # Start the neural network with its input-layer.
    encoder_input = Input(shape=(None, ), name='encoder_input')
    
    # Connect the embedding-layer.
    encoder_embeddings = Embedding(input_dim=100000, output_dim=300, name='encoder_embedding')(encoder_input)

    # Connect all the GRU-layers.
    encoder_layer_1 = GRU(latent_dim, name='encoder_layer_1', return_sequences=True)(encoder_embeddings)
    encoder_layer_2 = GRU(latent_dim, name='encoder_layer_2', return_sequences=True)(encoder_layer_1)

    # This is the output of the encoder.
    encoder_output = GRU(latent_dim, name='encoder_output', return_sequences=False)(encoder_layer_2)
    
    return encoder_input, encoder_output

In [5]:
encoder_input, encoder_output = connect_encoder()

Instructions for updating:
Colocations handled automatically by placer.


In [6]:
model_encoder = Model(inputs=[encoder_input], outputs=[encoder_output])

In [7]:
plot_model(model_encoder)

# Decoder

In [8]:
decoder_initial_state = Input(shape=(latent_dim,), name='decoder_initial_state')

In [9]:
def connect_decoder(initial_state):
    # Start the decoder-network with its input-layer.
    decoder_input = Input(shape=(None, ), name='decoder_input')

    # Connect the embedding-layer.
    decoder_embedding = Embedding(input_dim=100000, output_dim=300,  name='decoder_embedding')(decoder_input)
    
    # Connect all the GRU-layers.
    decoder_layer_1 = GRU(latent_dim, name='decoder_layer_1', return_sequences=True)(decoder_embedding, initial_state=initial_state)
    decoder_layer_2 = GRU(latent_dim, name='decoder_layer_2', return_sequences=True)(decoder_layer_1, initial_state=initial_state)
    decoder_layer_3 = GRU(latent_dim, name='decoder_gru3', return_sequences=True)(decoder_layer_2, initial_state=initial_state)

    # Connect the final dense layer that converts to
    # one-hot encoded arrays.
    decoder_output = Dense(100000, activation='linear', name='decoder_output')(decoder_layer_3)
    
    return decoder_input, decoder_output

In [11]:
decoder_input, decoder_output = connect_decoder(decoder_initial_state)

In [12]:
model_decoder = Model(inputs=[decoder_input, decoder_initial_state], outputs=[decoder_output])

In [13]:
plot_model(model_decoder)

# Model Input and Output

In [14]:
decoder_input, decoder_output = connect_decoder(encoder_output)

In [15]:
model_train = Model(inputs=[encoder_input, decoder_input], outputs=[decoder_output])

In [16]:
plot_model(model_train)